In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math

K_number = "5"
model1 = tf.keras.models.load_model(r"F:\Courses\Paper\Mine\Paper3\Results new\Model\LSTM-K1-K_max10-sequence30-delta1.h5")

mean = np.array([ 0.82279619,   1.22980571,   3.9329219,    0.30791619,   0.06652952,
                    -31.8272,      0.80288,  1.19558095,    3.76673524,   0.28387048,
                  0.16338286,  -0.37188571,  0.81583238,    1.22253714,   3.61266286,
                  0.24752762,   0.07504762,  31.2256    ])

STD = np.array([0.38184109,  0.72897371,  2.7642141,   0.46163168,  0.24920543, 17.25409012,
                0.39782371,  0.74267694,  2.72513013,  0.45087474,  0.39428699, 10.23517679,
                0.38762083,  0.73576464,  2.64014798,  0.43157583,  0.29355984, 17.34107153])

#Right Hand   #Hand  X=Positions[4] Y=Positions[5] Z=Positions[6]   #Elbow X=Positions[7] Y=Positions[8] Z=Positions[9] #Shoulder X=Positions[10] Y=Positions[11] Z=Positions[12]
#Head X=Positions[1] Y=Positions[2] Z=Positions[3]
#Left Hand   #Hand  X=Positions[13] Y=Positions[14] Z=Positions[15]   #Elbow X=Positions[16] Y=Positions[17] Z=Positions[18] #Shoulder X=Positions[19] Y=Positions[20] Z=Positions[21]

def pointing(Positions):
    if abs(Positions[5]-Positions[11])<0.1 or abs(Positions[14]-Positions[20])<0.1:
        if Positions[5]-Positions[8]<0.1 or Positions[14]-Positions[17]<0.1:
            return 1
    return 0

def pointingDirection(Positions):
    if abs(Positions[5]-Positions[11])<0.2:
        if Positions[6]-Positions[12]<0.2:
            return 1
        
    if abs(Positions[14]-Positions[20])<0.2:
        if Positions[15]-Positions[21]<0.2:
            return 2
    return 0


def CrossedArms(Positions):
    if abs(Positions[4]-Positions[10])>0.1 and abs(Positions[13]-Positions[19])>0.1:
        if abs(Positions[5]-Positions[8])<0.1 and abs(Positions[14]-Positions[17])<0.1:
            return 1
    return 0

def Conversation(Positions):
    if abs(Positions[10]-Positions[19])<0.2:
        return 1
    return 0


def waving(Positions):
    if Positions[5]-Positions[8]>0.2 or Positions[14]-Positions[17]>0.2:
        return 1
    return 0

def talking(Audio):
    for i in range(30):
        if Audio[i,0,1]<-5:
            return 3
        elif Audio[i,0,1]>-5 and Audio[i,0,1]<5:
            return 2
        elif Audio[i,0,1]>5:
            return 1
    return 0

def Enter(Positions):
    unique_positions = np.unique(Positions)
    std = np.std(unique_positions)
    if std > 0.5:
        # Mask the zeros
        A = abs(Positions)
        masked_arr = np.ma.masked_equal(A, 0)
        min_val = masked_arr.min()
        min_index = masked_arr.argmin()
        max_val = masked_arr.max()
        max_index = masked_arr.argmax()
        if max_index - min_index<0:
            return 1
    return 0

def Walk(Positions):
    unique_positions = np.unique(Positions)
    std = np.std(unique_positions)
    if std > 10:
        return 1
    return 0

def TT(Positions):
    Positions_new = np.zeros((Positions.shape[0], Positions.shape[1], Positions.shape[2]))
    Persons = np.zeros(3)
    for i in range(3):
        Persons[i] = Positions[-1,i,0]
        
    for j in range(30):
        for i in range(3):
            if np.array_equal(Persons[i], Positions[j,i,0]):
                Positions_new[j,i] = np.array(Positions[j,i])
            else:
                diff = np.setdiff1d(Positions[j,:,0], Persons)
                #diff_indices = np.where(np.isin(Positions[j,:,0], diff))
                if diff.shape[0]>0:
                    idx = np.where(Persons == 0)[0][0]
                    Persons[idx] = diff
                    
                if Persons[i] in Positions[j,:,0]:
                    if Persons[i]==0:
                        continue
                    index = np.where(Positions[j,:,0] == Persons[i])[0][0]
                    Positions_new[j,i] = np.array(Positions[j,index])
                
    return Positions_new

def RR(Positions):
    check = np.zeros(3)
    Positions_new = np.zeros((Positions.shape[0], Positions.shape[1], Positions.shape[2]))
#    for i in range(3):
#        if EL(Positions[:,i,1]) > 0:##
#            check[i] = 1
    if np.array_equal(check, np.zeros(3)):
        #print(Positions[-1])
        for i in range(30):
            for j in range(3):
                if Positions[i,j,1] > 0.4: #Person Number 1
                    Positions_new[i,0] = Positions[i,j]
                elif Positions[i,j,1] < 0.4 and Positions[i,j,1] > -0.4 and Positions[i,j,1] != 0: #Person Number 2
                    #print("Hi")
                    Positions_new[i,1] = np.array(Positions[i,j])
                    
                elif Positions[i,j,1] < -0.4: #Person Number 3
                    Positions_new[i,2] = Positions[i,j]
        #print(Positions_new[-1])
#     else:
#         index = np.where(check == 1)[0]
#         for i in range(30):
#             for j in range(4):
#                 if j == index:
#                     continue
#                 if Positions[i,j,1]>0.6: #Person Number 1
#                     Positions_new[i,0] = Positions[i,j]
#                 elif Positions[i,j,1]<0.6 and Positions[i,j,1]>0: #Person Number 2
#                     Positions_new[i,1] = Positions[i,j]
#                 elif Positions[i,j,1]<0 and Positions[i,j,1]>-0.6: #Person Number 3
#                     Positions_new[i,2] = Positions[i,j]
#                 elif Positions[i,j,1]<-0.6: #Person Number 4
#                     Positions_new[i,3] = Positions[i,j]
        
#         for i in range(30):
#             if index == 0 or index == 1:
#                 if Positions_new[i,1,0] > 0: #meaning there is a person in position 2 
#                     Positions_new[i ,0] = Positions[i,index]
#                 else:
#                     Positions_new[i ,1] = Positions[i,index]
#             elif index == 2 or index == 3:
#                 if Positions_new[i,2,0] > 0: #meaning there is a person in position 2 
#                     Positions_new[i ,3] = Positions[i,index]
#                 else:
#                     Positions_new[i ,2] = Positions[i,index]
                    
    return Positions_new

def Create_Input(Positions, Audio):
    Input = np.zeros((1,30,18))
    for i in range(30):
        for j in range(3):
            if Positions[i,j,0]>0 and Positions[i,j,3]<3: #Persons who are in the screen
            #if Positions[i,j,0]>0: #Person Number one
                Input[0,i,6*j] = 1
                if Positions[i,j,3]>2.1: #Far or Near
                    Input[0,i,6*j+1] = 2
                else:
                    Input[0,i,6*j+1] = 1

                if waving(Positions[i,j]) == 1:
                    Input[0,i,6*j+2] = 4
                elif pointing(Positions[i,j]) == 1:
                    Input[0,i,6*j+2] = 8
                elif CrossedArms(Positions[i,j]) == 1:
                    Input[0,i,6*j+2] = 5
                elif Conversation(Positions[i,j]) == 1:
                    Input[0,i,6*j+2] = 6
                elif Walk(Positions[:,j,1]) == 1:
                    Input[0,i,6*j+2] = 2
                elif Walk(Positions[:,j,3]) == 1:
                    Input[0,i,6*j+2] = 3
                elif Enter(Positions[:,j,1]) == 1:
                    Input[0,i,6*j+2] = 7
                else:
                    Input[0,i,6*j+2] = 1

                if talking(Audio) == j+1:
                    Input[0,i,6*j+3] = 1

                

                # Calculate the arctangent of a value
                
                atan_value = -2*math.atan(Positions[i,j,1]/Positions[i,j,3])
                Input[0,i,6*j+5] = math.degrees(atan_value)

                    
    for j in range(3):
        if Input[0,i,6*j+2] == 8:
              
            arr = np.array([0, 6, 12])

            # Find the indexes of occurrences of 6
            indexes = np.where(arr == 6*j)[0]

            # Delete all occurrences of 6
            arr = np.delete(arr, indexes)

            indexes = np.where(Input[0,i,arr] > 0)[0]
              
            if indexes.shape[0] == 1:
                Input[0,i,6*indexes[0]+4] = 1
                      
            if indexes.shape[0] == 2:
                if j==0 or j==2:
                    Input[0,i,6*1+4] = 1
                elif j==1:
                    if pointingDirection(Positions[i,j])==1:
                        Input[0,i,6*0+4] = 1
                    elif pointingDirection(Positions[i,j])==2:
                        Input[0,i,6*2+4] = 1

                        
    return Input

def Normalize(Input, Mean, STD):
    Input_normalized = np.zeros((Input.shape[0], Input.shape[1], Input.shape[2]))
    for i in range(Input.shape[1]):
        Input_normalized[0,i] = (np.array(Input[0,i])-np.array(mean))/np.array(STD)
    return Input_normalized

def Predict(y_pred, Input):
    Persons = np.zeros(3)
    for i in range(3):
        Persons[i] = np.array(Input[0,-1,6*i])
    indices = np.where(Persons == 1)[0]
    #y_pred[0,3] = 0
    for i in range(3):
        max_index = np.argmax(y_pred)
        if max_index in indices:
            return max_index + 1
        else:
            y_pred[0,max_index] = 0
    return 0

input = np.zeros((1,30,18))
Positions = np.zeros((30,3,22), dtype="float")
Audio = np.zeros((30,1,2), dtype="float")

for i in range(30):
    with open('F:\Courses\Paper\Mine\Paper3\Results new/Pos.txt') as file:
        B = file.read()
        while len(B)==0:
            B = file.read()
        array_1d = np.array([float(x.strip()) for x in B.split(',')])
        # Reshape the 1D array to a 3x22 array
        array_2d = array_1d.reshape(3, 22)
        Positions[i] = array_2d
    with open('F:\Courses\Paper\Mine\Paper3\Results new/Audio.txt') as file:
        B = file.read()
        while len(B)==0:
            B = file.read()
        array_1d = np.array([float(x.strip()) for x in B.split(',')])
        # Reshape the 1D array to a 1x2 array
        array_2d = array_1d.reshape(1, 2)
        Audio[i] = array_2d
        



                    


In [2]:
while True:
    Positions[0:29,:,:] = np.array(Positions[1:30,:,:])
    Audio[0:29,:,:] = np.array(Audio[1:30,:,:])
    with open('F:\Courses\Paper\Mine\Paper3\Results new/Pos.txt') as file:
        B = file.read()
        while len(B)==0:
            B = file.read()
        array_1d = np.array([float(x.strip()) for x in B.split(',')])
        # Reshape the 1D array to a 3x22 array
        array_2d = array_1d.reshape(3, 22)
        Positions[-1] = array_2d
            
    with open('F:\Courses\Paper\Mine\Paper3\Results new/Audio.txt') as file:
        B = file.read()
        while len(B)==0:
            B = file.read()
        array_1d = np.array([float(x.strip()) for x in B.split(',')])
        # Reshape the 1D array to a 1x2 array
        array_2d = array_1d.reshape(1, 2)
        Audio[-1] = array_2d
        
    Positions = TT(Positions)
    positions2 = RR(Positions)
    Input = Create_Input(positions2, Audio)
    Input_normalized = Normalize(Input, mean, STD) 
    y_pred = model1.predict(Input_normalized, verbose=0)
    Person_look = Predict(y_pred, Input)
    if Person_look>0:
        #print(positions2[-1])
        print(Person_look)
        #print(Input[0,-1,2+(Person_look-1)*6])
        print(Input[0,-1,5+(Person_look-1)*6]*-1, flush=True)
        #if Input[0,-1,2+(Person_look-1)*6]>0:
        #    print(Input[0,-1,5+(Person_look-1)*7]*-1+1000, flush=True)
        #else:
        #    print(Input[0,-1,5+(Person_look-1)*7]*-1, flush=True)
            

1
1.0
1
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
8.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
4.0
2
8.0
2
1.0
2
5.0
2
5.0
2
8.0
2
8.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
5.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
2
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.0
1
6.


KeyboardInterrupt



In [ ]:
Positions.shape

In [ ]:
Audio.shape

In [ ]:
Positions[0,0,1]<0.4

In [ ]:
positions2

In [ ]:
y_pred

In [3]:
Input[-1]

array([[  1.        ,   1.        ,   6.        ,   0.        ,
          0.        , -59.68185495,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  1.        ,   1.        ,   6.        ,   0.        ,
          0.        , -59.68185495,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  1.        ,   1.        ,   6.        ,   0.        ,
          0.        , -59.68185495,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  1.        ,   1.        ,   6.        ,   0.        ,
          0.        , -59.68185495,   0.        ,   0.   